⚠️ NOTE:
This notebook is for research and validation only.
Production logic lives in the `src/` directory.

In [1]:
# Project path setup
import sys
from pathlib import Path

PROJECT_ROOT = Path(r"C:\Users\shubh\crypto-market-opportunity-engine")
sys.path.insert(0, str(PROJECT_ROOT))

print("✅ Project root added:", PROJECT_ROOT)


✅ Project root added: C:\Users\shubh\crypto-market-opportunity-engine


In [2]:
import pandas as pd
import numpy as np

from src.inference import load_model
# import src.config as config
import src.config as config
FEATURE_COLS = config.FEATURE_COLS


In [3]:
DATA_FEATURES = r"C:\Users\shubh\crypto-market-opportunity-engine\data\processed\BTCUSDT_5m_2025_features.parquet"

df = pd.read_parquet(DATA_FEATURES)

print("Shape:", df.shape)
print(df.head())


Shape: (105092, 26)
                         open      high       low     close    volume  \
open_time                                                               
2025-01-01 02:20:00  93765.42  93773.19  93702.61  93716.81  13.08729   
2025-01-01 02:25:00  93716.81  93742.52  93702.62  93742.52  11.93177   
2025-01-01 02:30:00  93742.51  93799.36  93742.51  93773.61  13.91987   
2025-01-01 02:35:00  93773.61  93880.00  93773.60  93879.99  12.32596   
2025-01-01 02:40:00  93879.99  93921.86  93855.17  93921.86  13.58807   

                                    close_time  quote_asset_volume  \
open_time                                                            
2025-01-01 02:20:00 2025-01-01 02:24:59.999999        1.226630e+06   
2025-01-01 02:25:00 2025-01-01 02:29:59.999999        1.118177e+06   
2025-01-01 02:30:00 2025-01-01 02:34:59.999999        1.305277e+06   
2025-01-01 02:35:00 2025-01-01 02:39:59.999999        1.156602e+06   
2025-01-01 02:40:00 2025-01-01 02:44:59.999999  

In [4]:
# load trained model
MODEL_NAME = "random_forest"   # best so far
VERSION = "1"

model = load_model(MODEL_NAME, VERSION)

print("✅ Model loaded:", MODEL_NAME, VERSION)


✅ Model loaded: random_forest 1


In [5]:
# generate prob row by row
df["prob_up"] = np.nan
batch_size = 100

for i in range(0, len(df), batch_size):
    X_batch = df.iloc[i:i+batch_size][FEATURE_COLS]
    df.iloc[i:i+batch_size, df.columns.get_loc("prob_up")] = (
        model.predict_proba(X_batch)[:, 1]
    )



In [6]:
# signal logic
BUY_THRESHOLD = 0.60
SELL_THRESHOLD = 0.40

def generate_signal(p):
    if p >= BUY_THRESHOLD:
        return "BUY"
    elif p <= SELL_THRESHOLD:
        return "SELL"
    else:
        return "HOLD"

df["signal"] = df["prob_up"].apply(generate_signal)

df["signal"].value_counts(normalize=True)


signal
HOLD    0.999467
BUY     0.000438
SELL    0.000095
Name: proportion, dtype: float64

In [7]:
# live style paper trading engine
INITIAL_CAPITAL = 100_000
RISK_PER_TRADE = 0.01
STOP_LOSS_PCT = 0.01
TAKE_PROFIT_PCT = 0.02
FEE_RATE = 0.0004

capital = INITIAL_CAPITAL
position = 0
entry_price = 0
position_size = 0

df["capital"] = capital
df["pnl"] = 0.0


In [8]:
# candle by candle simulation
for i in range(len(df)):
    price = df.iloc[i]["close"]
    signal = df.iloc[i]["signal"]

    # ENTER
    if signal == "BUY" and position == 0:
        risk_amount = capital * RISK_PER_TRADE
        stop_loss = price * (1 - STOP_LOSS_PCT)
        take_profit = price * (1 + TAKE_PROFIT_PCT)

        position_size = risk_amount / (price - stop_loss)
        entry_price = price
        position = 1

    # EXIT
    elif position == 1:
        if price <= stop_loss or price >= take_profit or signal == "SELL":
            pnl = (price - entry_price) * position_size
            pnl -= abs(pnl) * 2 * FEE_RATE

            capital += pnl
            df.iloc[i, df.columns.get_loc("pnl")] = pnl

            position = 0
            position_size = 0

    df.iloc[i, df.columns.get_loc("capital")] = capital


In [9]:
# live equity curve
df["equity_curve"] = df["capital"] / INITIAL_CAPITAL

df[["capital", "equity_curve"]].tail()


,capital,equity_curve
open_time,,
2025-12-31 23:35:00,125410.780822,1.254108
2025-12-31 23:40:00,125410.780822,1.254108
2025-12-31 23:45:00,125410.780822,1.254108
2025-12-31 23:50:00,125410.780822,1.254108
2025-12-31 23:55:00,125410.780822,1.254108


In [10]:
# live performance summary
trades = df[df["pnl"] != 0]

total_return = df["equity_curve"].iloc[-1] - 1
win_rate = (trades["pnl"] > 0).mean()
max_dd = (df["equity_curve"] / df["equity_curve"].cummax() - 1).min()

print(f"📈 Total Return: {total_return:.2%}")
print(f"🎯 Win Rate: {win_rate:.2%}")
print(f"📉 Max Drawdown: {max_dd:.2%}")
print(f"🔁 Trades: {len(trades)}")


📈 Total Return: 25.41%
🎯 Win Rate: 52.94%
📉 Max Drawdown: -3.26%
🔁 Trades: 34


In [11]:
# sanity checks
df[["close", "prob_up", "signal", "capital"]].tail(10)


,close,prob_up,signal,capital
open_time,,,,
2025-12-31 23:10:00,87689.75,0.493308,HOLD,125410.780822
2025-12-31 23:15:00,87651.88,0.490690,HOLD,125410.780822
2025-12-31 23:20:00,87675.47,0.486662,HOLD,125410.780822
2025-12-31 23:25:00,87675.44,0.488431,HOLD,125410.780822
2025-12-31 23:30:00,87671.00,0.489571,HOLD,125410.780822
2025-12-31 23:35:00,87652.47,0.492735,HOLD,125410.780822
2025-12-31 23:40:00,87685.31,0.494017,HOLD,125410.780822
2025-12-31 23:45:00,87690.61,0.492686,HOLD,125410.780822
2025-12-31 23:50:00,87641.14,0.489541,HOLD,125410.780822
